In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.00001

Inductance = 1#5.
R = 1.

solver = "ca"
# solver = "ca_canonical"
# solver = "ca_canonical_X"

# Ciropt problem

In [3]:
problem = co.douglas_rachford_splitting(mu, L_smooth, R, Inductance)
problem.obj = problem.b #+ problem.d

# bounds = {  'b': {"ub": 0.1, "lb": 0.1},
#             'h': {"ub": 1, "lb": 1},
#             'alpha': {"ub": 1, "lb": 1},
#             'beta': {"ub": 1, "lb": 1},}

# res, sol = problem.solve(solver=solver, bounds=bounds, verbose=True)[:2]
res, sol = problem.solve(solver=solver, verbose=True)[:2]
ca_vars = problem.vars

dim_G=9, dim_F=8
opts={'ipopt.max_iter': 50000}

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     3770
Number of nonzeros in inequality constraint Jacobian.:       59
Number of nonzeros in Lagrangian Hessian.............:      214

Total number of variables............................:      152
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints........

In [4]:
res

{'b': -5.440530963737621e-10,
 'h': 2.75548423089315e-10,
 'alpha': -0.01518446465637779,
 'beta': 0.007644362315037343}

# PEP verification

In [5]:
problem = co.douglas_rachford_splitting(mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 9x9
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (0 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 12 scalar constraint(s) ...
		 function 1 : 12 scalar constraint(s) added
		 function 2 : Adding 12 scalar constraint(s) ...
		 function 2 : 12 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: -2.3053307988780658e-09
diff=-2.3053307988780658e-09


In [6]:
import PEPit
import PEPit.functions as pep_func
from PEPit.primitive_steps import proximal_step as pep_proximal_step

import numpy as np
import cvxpy as cp

In [7]:
# verification mode: PEP
problem = PEPit.PEP()
package = pep_func
proximal_step = pep_proximal_step

L_smooth = 1
mu = 0.1

rho = 1
f = co.define_function(problem, mu, L_smooth, package)
g = co.define_function(problem, mu, L_smooth, package)

x_star, y_star, _ = (f + g).stationary_point(return_gradient_and_function_value=True)
z0 = problem.set_initial_point()

x1, grad_f_1, f_1 = proximal_step(z0, f, 1/rho)
z1, grad_g_1, g_1 = proximal_step(x1, g, 1/rho)
y1 = rho * (x1 - z1)

x2, _, _ = proximal_step(z1 - (1/rho) * y1, f, 1/rho)
z2, _, _ = proximal_step(x2 + (1/rho) * y1, g, 1/rho)
y2 = y1 + rho * (x2 - z2)

E_1 = (1/rho) * (y1 - y_star)**2 + rho * (z1 - x_star)**2
E_2 = (1/rho) * (y2 - y_star)**2 + rho * (z2 - x_star)**2
Delta_1 = rho * (x2 - z2)**2 + rho * (z2 - z1)**2

problem.set_performance_metric(E_2 - (E_1 - Delta_1))

In [8]:
diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 7x7
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (0 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: unbounded (solver: MOSEK); optimal value: inf


UserWarning: PEPit didn't find any nontrivial worst-case guarantee. It seems that the optimal value of your problem is unbounded.

In [ ]:
from PEPit.functions import SmoothConvexFunction
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step

# Instantiate PEP
problem = PEPit.PEP()
n = 1
L=1; alpha=1; theta=1; verbose=1

# Declare a convex and a smooth convex function.
func1 = problem.declare_function(ConvexFunction)
func2 = problem.declare_function(SmoothConvexFunction, L=L)
# Define the function to optimize as the sum of func1 and func2
func = func1 + func2

# Start by defining its unique optimal point xs = x_* and its function value fs = F(x_*)
xs = func.stationary_point()
fs = func(xs)

# Then define the starting point x0 of the algorithm and its function value f0
x0 = problem.set_initial_point()

# Compute n steps of the Douglas-Rachford splitting starting from x0
x = [x0 for _ in range(n)]
w = [x0 for _ in range(n + 1)]
for i in range(n):
    x[i], _, _ = proximal_step(w[i], func2, alpha)
    y, _, fy = proximal_step(2 * x[i] - w[i], func1, alpha)
    w[i + 1] = w[i] + theta * (y - x[i])

# Set the initial constraint that is the distance between x0 and xs = x_*
problem.set_initial_condition((x[0] - xs) ** 2 <= 1)

# Set the performance metric to the final distance to the optimum in function values
problem.set_performance_metric((func2(y) + fy) - fs)

# Solve the PEP
pepit_verbose = max(verbose, 0)
pepit_tau = problem.solve(verbose=pepit_verbose)
print(pepit_tau)

(PEPit) Setting up the problem: size of the main PSD matrix: 6x6
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (1 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 2 scalar constraint(s) ...
		 function 1 : 2 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 0.2500000073849442
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -9.66e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply 